In [ ]:
# coding:utf-8
from hashlib import sha1
from flask import Flask, request
import time
import urllib.request
import json
import wxfunc

token = 'Flashield'
appid = 'wx52d8b7b32451e895'
appsec = 'fb9eeaf21b92ceed9b8200c44fabea6d'
AccessToken = {'access_token': '12_46y9bFrNQlGBfJV1XkYMrrIc_Ab55sB0KZaYngky7dnUIZsx_40b50HSwzJry-ii7yvpaYWVgoF9Pnd_oT6V6EOJcUt1nFRiPbCpUJD1UibSVUaUftHDm80Tksjw2lhRoTMgCaLUEQH6k4vQDHPgAFALPX', 'live_time': 1534768234}

app = Flask(__name__)

def get_token(appid,appsec):
    url = 'https://api.weixin.qq.com/cgi-bin/token?grant_type=client_credential&appid={}&secret={}'.format(appid,appsec)    
    if int(AccessToken['live_time']) < int(time.time()):
        opener = urllib.request.build_opener()
        request = opener.open(url)
        get_acc_token = request.read().decode(encoding='UTF8',errors='ignore')
        AccessToken['access_token'] = json.loads(get_acc_token)['access_token']
        AccessToken['live_time'] = int(time.time())+json.loads(get_acc_token)['expires_in']-60
    print(AccessToken['access_token'],AccessToken['live_time'])
    return AccessToken['access_token']

def get_update(token, timestamp, nonce):
    arguments = ''
    for k in sorted([token, timestamp, nonce]):
        arguments = arguments + str(k)
    m = sha1()
    m.update(arguments.encode('utf8'))
    return m.hexdigest()

def check_signature():
    signature = request.args.get('signature', '')
    timestamp = request.args.get('timestamp', '')
    nonce = request.args.get('nonce', '')
    print(signature)
    check = get_update(token, timestamp, nonce)
    return True if check == signature else False

@app.route('/wx', methods=['GET', 'POST'])
def weixinInterface():
    if check_signature:
        if request.method == 'GET':
            echostr = request.args.get('echostr', '')
            return echostr
        elif request.method == 'POST':
            data = request.data
            print("data from Weixin:{}".format(data))
            wxfunc.sms_msg(data)
            msg = wxfunc.parse_xml(data)
            get_token(appid,appsec)
        return msg
    else:
        return 'signature error'
    
@app.route('/')
def index():
    return ""

if __name__ == '__main__':
    app.run(host='0.0.0.0',port=9999)

 * Running on http://0.0.0.0:9999/ (Press CTRL+C to quit)


data from Weixin:b'<xml><ToUserName><![CDATA[gh_730770a41cd7]]></ToUserName>\n<FromUserName><![CDATA[o9IEI1c3I7kafdiTK5RrQUTYXpYg]]></FromUserName>\n<CreateTime>1577426978</CreateTime>\n<MsgType><![CDATA[text]]></MsgType>\n<Content><![CDATA[test]]></Content>\n<MsgId>22583322227880903</MsgId>\n</xml>'


168.235.99.33 - - [27/Dec/2019 14:09:40] "POST /wx?signature=6eba0b3e0b2f80b29ac27758a14034b2e34f4b44&timestamp=1577426978&nonce=848814230&openid=o9IEI1c3I7kafdiTK5RrQUTYXpYg HTTP/1.0" 200 -


send_msg:{'MsgType': 'text', 'Content': '测试', 'FromUserName': 'gh_730770a41cd7', 'ToUserName': 'o9IEI1c3I7kafdiTK5RrQUTYXpYg'}
datas:<xml>
        <FromUserName>gh_730770a41cd7</FromUserName>
        <ToUserName>o9IEI1c3I7kafdiTK5RrQUTYXpYg</ToUserName>
        <CreateTime>1577426980</CreateTime>
        <MsgType>text</MsgType>
        <Content>测试</Content>
        </xml>
28_LGKDg7GLfUjE3iRjGHw6s4SvOfKUisIFqlXrlg99x1wSWuTVFhNzb17eoERLwCfR4pi_jReHxRqudavXr5dr9Ki-jJ8ejxjN4gV2q1fMuQdeBrJ7ziPpha_8WqDqqTACzKqv6OAHIZgojSn2NRQcABAVZY 1577434120
